In [2]:
from agents.agents import agent_generation
from agents.agents import middleware_generation
from agents.query_support import clean_text
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
import json
import pandas as pd
from langchain.tools import tool
from typing import List, Dict, Tuple


In [3]:
from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# === Claves principales ===
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# === LangSmith config ===
LANGSMITH_TRACING_V2 = os.getenv("LANGSMITH_TRACING_V2", "true")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT", "RAG-Municipios")

os.environ["LANGSMITH_TRACING_V2"] = LANGSMITH_TRACING_V2
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"



# Agente de Análisis

In [4]:
from agents.analysis_agent.helpers import recomendar_municipios_por_cultivo_1
from agents.analysis_agent.helpers import recomendar_cultivos_por_municipio_1
from agents.analysis_agent.helpers import cultivos_comunes_dict_1
from agents.analysis_agent.helpers import top_municipios_por_cultivo
from agents.analysis_agent.helpers import top_cultivos_por_municipio
from agents.analysis_agent.prompts import rewriter_prompt_generator
from agents.analysis_agent.prompts import analyst_prompt
from langchain.messages import AIMessage

In [5]:
with open("results/lista_cultivos.json", "r", encoding="utf-8") as f:
    lista_cultivos = json.load(f)

df_similarity = pd.read_parquet("results/similarity_matrix.parquet")
df_confidence = pd.read_parquet("results/confianza_matrix.parquet")
suelo = pd.read_parquet("results/soil.parquet")
with open("results/municipios.json", "r", encoding="utf-8") as f:
    municipios_data = json.load(f)
with open("results/cultivos.json", "r", encoding="utf-8") as f:
    cultivo_data = json.load(f) 
with open("results/dict_cvegeo.json", "r", encoding="utf-8") as f:
    dict_cvegeo = json.load(f)


In [27]:
lista_cultivos

['Avena forrajera en verde',
 'Brócoli',
 'Pastos y praderas',
 'Chile verde',
 'Fresa',
 'Maíz forrajero en verde',
 'Maíz grano',
 'Sorgo forrajero en verde',
 'Tomate rojo (jitomate)',
 'Tomate verde',
 'Triticale forrajero en verde',
 'Zanahoria',
 'Frijol',
 'Alfalfa',
 'Durazno',
 'Granada',
 'Membrillo',
 'Nopalitos',
 'Nuez',
 'Uva',
 'Agave',
 'Tuna',
 'Cebolla',
 'Coliflor',
 'Lechuga',
 'Acelga',
 'Betabel',
 'Chícharo',
 'Cilantro',
 'Col (repollo)',
 'Rábano',
 'Espárrago',
 'Calabacita',
 'Ejote',
 'Elote',
 'Espinaca',
 'Cacahuate',
 'Pepino',
 'Aguacate',
 'Guayaba',
 'Higo',
 'Lima',
 'Limón',
 'Manzana',
 'Naranja',
 'Pitahaya',
 'Ajo',
 'Camote',
 'Nopal forrajero',
 'Verdolaga',
 'Cebada forrajera en verde',
 'Pasto tapete',
 'Aceituna',
 'Girasol flor ',
 'Ciruela',
 'Clavel',
 'Flores',
 'Mandarina',
 'Haba verde',
 'Sandía',
 'Chabacano',
 'Palma de ornato',
 'Pera',
 'Col de bruselas',
 'Napa',
 'Avena grano',
 'Cebada grano',
 'Trigo grano',
 'Albahaca',
 'Meló

## Tools

In [6]:

@tool 
def recomendar_municipios_por_cultivo(cultivo: str, min_value: float = 4.0, top_k=3, min_score=0.70) -> dict:
    """
    Given a crop, identifies municipalities with similar agro-environmental conditions 
    suitable for its cultivation.  
    Focuses on finding **new potential areas (similares)** while also listing 
    **current producing municipalities (originales)** and how well they perform.  
    Uses a score = similarity*confidence and includes detailed soil feature vectors for each.

    Parameters:
      • cultivo — Crop name to analyze and find suitable municipalities for.  
      • min_value — Minimum cultivation value to consider a municipality as an “original” producer (default: 4.0).  
      • top_k — Number of top similar municipalities to include in the recommendation (default: 3).  
                 Helps prevent cases where no similar municipality meets the conditions.  
      • min_score — Minimum similarity-confidence score to keep a municipality as valid (default: 0.70).  
    """
    return recomendar_municipios_por_cultivo_1(cultivo_data, df_similarity, df_confidence, cultivo, suelo, min_value, top_k, min_score)

@tool
def recomendar_cultivos_por_municipio(cvegeo: str, top_n: int = 1, min_value: float = 4.0) -> dict:
    """
    Given a municipality (CVEGEO), identifies the most similar municipalities based on 
    similarity*confidence scores.  
    For each similar municipality, it provides:
      • **New crops** that are promising but not currently cultivated.  
      • **Shared crops** with their respective values in both municipalities and the difference.  
    Also includes detailed soil feature vectors for both the original and similar municipalities.

    Parameters:
      • cvegeo — Unique municipality identifier (CVEGEO) to use as the reference.  
      • top_n — Number of top similar municipalities to include in the analysis (default: 1).  
      • min_value — Minimum cultivation value required for a crop to be considered significant 
                    in the similarity and recommendation process (default: 4.0).  
    """
    return recomendar_cultivos_por_municipio_1(cvegeo, df_similarity, df_confidence, municipios_data, suelo, top_n, min_value)

@tool
def cultivos_comunes(cvegeo1: str, cvegeo2: str) -> dict:
    """
    Compares two municipalities (CVEGEO1 and CVEGEO2) and returns the crops they share in common,
    including their respective values and the performance difference.
    Numeric suitability values are mapped to qualitative labels (e.g., “Excelente”, “Regular”, “Malo”) for easier interpretation.
    """
    return cultivos_comunes_dict_1(municipios_data, cvegeo1, cvegeo2)

# ===========================
# Tool 1: Top municipios por cultivo
# ===========================
@tool
def top_municipios_cultivo(cultivo_name: str, N: int = 5) -> List[Dict]:
    """
    Devuelve los top N municipios para un cultivo específico según su valor.
    Nota: Solo devuelve datos de cultivo, no incluye interpretación ni inferencia del modelo.

    Ejemplo:
        top_municipios_cultivo("Manzana", 3)
        # Devuelve los 3 municipios con mayor producción o valor de cultivo "Manzana"

    Args:
        cultivo_name (str): Nombre del cultivo (ej. "Manzana")
        N (int): Número de top resultados a devolver (por defecto 5)

    Returns:
        list of dict: Top N municipios con su información y valor
    """
    return top_municipios_por_cultivo(cultivo_data, cultivo_name, N)

# ===========================
# Tool 2: Top cultivos por municipio
# ===========================
@tool
def top_cultivos_municipio(municipio_id: str, N: int = 5) -> List[Tuple[str, float]]:
    """
    Devuelve los top N cultivos para un municipio específico según su valor.
    Nota: Solo devuelve datos de cultivo, no incluye interpretación ni inferencia del modelo.

    Ejemplo:
        top_cultivos_municipio("01002", 3)
        # Devuelve los 3 cultivos más importantes en el municipio con CVEGEO "01002"

    Args:
        municipio_id (str): CVEGEO del municipio (ej. "01002")
        N (int): Número de top cultivos a devolver (por defecto 5)

    Returns:
        list of tuples: [(nombre_cultivo, valor), ...] ordenados de mayor a menor
    """
    return top_cultivos_por_municipio(municipios_data, municipio_id, N)


## Agents creation

In [146]:
rewriter_prompt = rewriter_prompt_generator(lista_cultivos, dict_cvegeo)
rewriter_agent = agent_generation(prompt=rewriter_prompt,
                                  model='gpt-5-nano',
                                  reasoning_effort='minimal'
                                  )
checkpointer_analyst = InMemorySaver()
middleware_analyst = middleware_generation(messages_to_keep=3)
analyst_agent = agent_generation(prompt=analyst_prompt,
                                model='gpt-5-nano',
                                reasoning_effort='minimal',
                                tools=[recomendar_municipios_por_cultivo,
                                       recomendar_cultivos_por_municipio,
                                       cultivos_comunes, top_municipios_cultivo,
                                       top_cultivos_municipio],
                                checkpointer=checkpointer_analyst,
                                middleware=[middleware_analyst]
                                )


In [ ]:

def analyst_chat(query):
    config: RunnableConfig = {"configurable": {"thread_id": "1"}}
    input_data = {"messages": [{"role": "user", "content": clean_text(query)}]}

    response = rewriter_agent.invoke(input_data)
    ai_content = next(
    (msg.content for msg in response['messages'] if isinstance(msg, AIMessage)),
    None)
    response_rewriter = ai_content
    #print(response_rewriter)
    model_messages = []
    for item in analyst_agent.stream(
    {"messages": [{"role": "user", "content": response_rewriter}]},
    config=config
    ):
        val = item.get("model")
        if val is not None:
            model_messages.append(val.get("messages")[0].content)

    model_message = "".join(model_messages)
    for k in model_message:
        print(k, end="", flush=True)


In [175]:
query = "Call the 2nd tool using 01002 and give me the results"
check = analyst_chat(query)

Resultado solicitado para el municipio 01002 (Asientos, Aguascalientes) usando la segunda herramienta recomendar_cultivos_por_municipio.

Contexto
- CVEGEO de referencia: 01002.
- Objetivo: identificar municipios similares y crops prometedores basados en la similitud agroambiental y las condiciones del suelo.
- Umbral: valores de cultivo >= 4.0 para considerar significativos.

Resultado de la consulta (recomendaciones y detalles)
1) Municipio similar destacado
- Similar: 16085 (Tangancícuaro, Michoacán)
- Métrica de similitud: similarity ≈ 0.807; confidence ≈ 99.4%; score ≈ 80.20
- Rasgos de suelo del similar: ALTITUD ≈ 2075 m; pH ≈ 6.86; contenidos de Ca, Mg, K, SNA, drenaje, entre otros, con condiciones moderadamente a ligeramente ácidas o neutras y buen drenaje.

2) Cultivos nuevos recomendados para 01002 (según el similar y criterios de valor ≥ 4.0)
- Frijol (value 4.0)
- Alfalfa (4.0)
- Durazno (5.0)
- Uva (4.0)
- Ejote (4.0)
- Manzana (5.0)
- Ajo (4.0)
- Nopal forrajero (5.0)

3)

In [164]:
from langchain.messages import HumanMessage
show = False
print(show)
for item in check:
    if isinstance(item, HumanMessage) or isinstance(item, ToolMessage):
            show = True # A partir de este punto empezamos a mostrar
            continue  

    if show:
        print(item.content, end="", flush=True)

False


In [60]:
val = check[4].get("model")
if val is not None:
    print(val.get("messages")[0].content,end="", flush=True)

Resumen de hallazgos
- Municipio de referencia: 01002 (Asientos, Aguascalientes). 
- Municipio similar sugerido: 16085 (Tangancícuaro, Michoacán) con alta similitud de condiciones agro-ambientales y buena confianza de relevancia para el cultivo.
- Cobertura de suelos y clima relevantes: 
  - Similaridad: score 80.2 (alta) con confianza ~99.4%.
  - Suelos en semejanza: Tangancícuaro presenta altitud ~2075 m, pH ~6.86, CE ~0.60, Ca ~6.05, Mg ~5.84, y otros rasgos de textura y drenaje indicativos de suelos ligeramente ácidos a neutros y buena capacidad de retención de agua.
  - Suelo de Asientos (01002) tiene pH muy ácido en registro (≈4.40 en el detalle original de pH) y valores de CE bajos/moderados; diferencia notable en pH podría limitar ciertos cultivos si no se corrige.
- Cultivos nuevos sugeridos para 16085 (para evaluar viabilidad en origen y/o expansión): Frijol, Alfalfa, Durazno, Uva, Ejote, Manzana, Ajo, Nopal forrajero. Muchos de estos tienen valor original 4.0 o superior, con

In [44]:
check[4].get("model").get("messages")[0].content

'Resumen de hallazgos\n- Municipio de referencia: 01002 (Asientos, Aguascalientes). \n- Municipio similar sugerido: 16085 (Tangancícuaro, Michoacán) con alta similitud de condiciones agro-ambientales y buena confianza de relevancia para el cultivo.\n- Cobertura de suelos y clima relevantes: \n  - Similaridad: score 80.2 (alta) con confianza ~99.4%.\n  - Suelos en semejanza: Tangancícuaro presenta altitud ~2075 m, pH ~6.86, CE ~0.60, Ca ~6.05, Mg ~5.84, y otros rasgos de textura y drenaje indicativos de suelos ligeramente ácidos a neutros y buena capacidad de retención de agua.\n  - Suelo de Asientos (01002) tiene pH muy ácido en registro (≈4.40 en el detalle original de pH) y valores de CE bajos/moderados; diferencia notable en pH podría limitar ciertos cultivos si no se corrige.\n- Cultivos nuevos sugeridos para 16085 (para evaluar viabilidad en origen y/o expansión): Frijol, Alfalfa, Durazno, Uva, Ejote, Manzana, Ajo, Nopal forrajero. Muchos de estos tienen valor original 4.0 o super

# Agente Supervisor

In [3]:
from agents.overview_multi_agent.helpers import download_embeddings
from langchain_pinecone import PineconeVectorStore
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool
from agents.overview_multi_agent.prompts import context_prompt
from agents.overview_multi_agent.prompts import web_prompt
from agents.overview_multi_agent.prompts import supervisor_prompt

In [4]:
embedding = download_embeddings()


In [5]:
# embbeded each chunk and upsert the embeddings into your pinecone index
index_name = "research-assistant"

vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [6]:

# RAG tool
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information from internal documentation or PDFs.

    Use this tool when the user asks about technical or implementation details
    within the project. It performs a semantic similarity search in the
    document embeddings to extract the most relevant passages.

    Example queries:
    - "Which library was used for VietorisRips?"
    - "Was DVC used for reproducibility?"
    - "Explain how GPU support was integrated with giotto-tda."
    """
    retrieved_docs = vector_store.similarity_search(query, k=2)
    if not retrieved_docs:
        return "No relevant context found in the documentation.", []
    serialized = "\n\n".join(
        f"📘 **Source:** {doc.metadata}\n{doc.page_content}" for doc in retrieved_docs
    )
    return serialized, retrieved_docs
# DuckDuckGo tool
@tool
def duckduckgo_search(query: str) -> str:
    """Search the web using DuckDuckGo.

    Use this tool for general information retrieval from online sources such as news,
    research sites, and Wikipedia. Limit results to the top 3 most relevant hits.

    Example queries:
    - "Recent research on persistent homology"
    - "Farms cultivating lettuce near Aguascalientes"
    """
    results = DuckDuckGoSearchRun().run(query)
    if not results:
        return "No results found."
    top_results = results[:3]
    return "\n".join(f"🔗 {r['title']}: {r['href']}" for r in top_results)

# Crear el client con Taviliy MCP
client = MultiServerMCPClient(
    {
        "tavily": {
            "transport": "streamable_http",
            "url": "https://mcp.tavily.com/mcp",
            "headers": {"Authorization": f"Bearer {TAVILY_API_KEY}"}
        }
    }
)
# Obtener todas las tools
tools_mcp = await client.get_tools()


## Agents Creation

### Context Agent

In [16]:
context_agent = agent_generation(prompt=context_prompt,
                 model="gpt-5-nano",
                 reasoning_effort="medium",
                 tools=[retrieve_context])

### Web Agent

In [17]:
web_agent = agent_generation(prompt=web_prompt,
                             model="gpt-5-nano",
                             reasoning_effort="low",
                             tools=[duckduckgo_search]+tools_mcp)


## Agents as tools

In [18]:
@tool
def context_retriever(request: str) -> str:
    """Retrieve answers from the project documentation."""
    result = context_agent.invoke({"messages": [{"role": "user", "content": request}]})
    response = result["messages"][-1] if isinstance(result, dict) else result
    return response.content if hasattr(response, "content") else str(response)


@tool
def web_retriever(request: str) -> str:
    """Retrieve information from the web using Tavily, DuckDuckGo, or Wikipedia MCP."""
    result = web_agent.invoke({"messages": [{"role": "user", "content": request}]})
    response = result["messages"][-1] if isinstance(result, dict) else result
    return response.content if hasattr(response, "content") else str(response)

## Creation of Supervisor Agent 

In [19]:
checkpointer_supervisor = InMemorySaver()
middleware_supervisor = middleware_generation(messages_to_keep=2)
supervisor_agent = agent_generation(prompt=supervisor_prompt,
                                model='gpt-5-nano',
                                reasoning_effort='medium',
                                tools=[context_retriever,
                                       web_retriever
                                       ],
                                checkpointer=checkpointer_supervisor,
                                middleware=[middleware_supervisor]
                                )

In [20]:
def supervisor_chat(query):
    config: RunnableConfig = {"configurable": {"thread_id": "2"}}
    query_cleaned = clean_text(query) 
    #print(response_rewriter)
    for token, metadata in supervisor_agent.stream(
        {"messages": [{"role": "user", "content": query_cleaned}]},
        stream_mode="messages", config=config
    ):
        print(token.content, end="", flush=True)

In [ ]:
query_2 = "Tell me how the persistent diagramas were calculated in the project and mention if it was used GPU"
supervisor_chat(query_2)

- How the persistence diagrams were calculated:
  - Data representation: Indicator features (climatic, edaphological, land-use) are collected per municipality and used to form a point cloud (after any required normalization).
  - Filtration: A Vietoris–Rips filtration is built from this point cloud to capture the topological features across scales.
  - Core computation: Persistent homology is computed with a GPU-accelerated adaptation of ripser++ serving as the back-end.
  - Output: Persistence diagrams (birth, death pairs) are produced for each homology dimension (H0, H1, and optionally H2). H0 diagrams may include points with infinite death (represented as infinity) corresponding to components that persist to the end of the filtration.

- GPU usage:
  - Yes. The workflow explicitly uses a GPU-accelerated ripser++ backend to speed up the computation of persistence diagrams.

- Reproducibility and traceability (context in the project):
  - The process is integrated into a reproducible,